In [4]:
import os
import re
import vrplib
import random
import matplotlib.pyplot as plt
import numpy as np
from collections import deque

# 准备工作

In [2]:
#按文件名排序加载数据
def read_all_instances(root_folder, ending='.tsp'):  
    instances = []  
    
    def extract_k_number(file_name):  
        match = re.search(r'k(\d+)', file_name)  
        if match:  
            return int(match.group(1))
        return float('inf')  

    file_names = sorted(  
        [file_name for file_name in os.listdir(root_folder) if file_name.endswith(ending)],  
        key=extract_k_number 
    )   
    for file_name in file_names:  
        instance = vrplib.read_instance(str(os.path.join(root_folder, file_name))) 
        if instance:  
            instances.append(instance)  
            print(f'Successfully read {file_name}')  
        else:
            print(f'Failed to read {file_name}')  
    
    return instances  

In [5]:
root_folder = './data/cvrp/new_data/'  
cvrp_instances = read_all_instances(root_folder, ending='.vrp')

Successfully read A-n34-k5.vrp
Successfully read A-n32-k5.vrp
Successfully read A-n36-k5.vrp
Successfully read A-n33-k5.vrp
Successfully read A-n37-k5.vrp
Successfully read A-n38-k5.vrp
Successfully read A-n39-k5.vrp
Successfully read A-n39-k6.vrp
Successfully read A-n37-k6.vrp
Successfully read A-n33-k6.vrp
Successfully read A-n44-k6.vrp
Successfully read A-n45-k6.vrp
Successfully read A-n53-k7.vrp
Successfully read A-n46-k7.vrp
Successfully read A-n54-k7.vrp
Successfully read A-n45-k7.vrp
Successfully read A-n48-k7.vrp
Successfully read A-n62-k8.vrp
Successfully read A-n65-k9.vrp
Successfully read A-n69-k9.vrp
Successfully read A-n61-k9.vrp
Successfully read A-n64-k9.vrp
Successfully read A-n55-k9.vrp
Successfully read A-n60-k9.vrp
Successfully read A-n63-k9.vrp
Successfully read A-n63-k10.vrp
Successfully read A-n80-k10.vrp


In [6]:
# Function to visualize solution
def plot_CVRP_solution(routes, node_coords):
    plt.figure(figsize=(10, 8))
    
    # Plot nodes
    for i, (x, y) in enumerate(node_coords):
        plt.scatter(x, y, c='blue' if i == 0 else 'red')
        plt.text(x, y, f'{i}', fontsize=9, ha='right')
    
    # Plot routes
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for vehicle, route in enumerate(routes):
        route_coords = [node_coords[0]] + [node_coords[node] for node in route] + [node_coords[0]]
        x_coords, y_coords = zip(*route_coords)
        plt.plot(x_coords, y_coords, c=colors[vehicle % len(colors)], label=f'Vehicle {vehicle + 1}')
    
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Vehicle Routing Problem Solution')
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to test vehicle capacity constraint
def test_capacity_constraint(routes, demands, vehicle_capacity):
    for vehicle, route in enumerate(routes):
        total_demand = sum(demands[node] for node in route)
        if total_demand > vehicle_capacity:
            print(f"Vehicle {vehicle + 1} exceeds capacity: {total_demand} > {vehicle_capacity}")
        else:
            print(f"Vehicle {vehicle + 1} is within capacity: {total_demand} <= {vehicle_capacity}")


# Function to calculate route distance
def calculate_route_distance(route, distance_matrix):
    distance = 0
    if route:
        distance += distance_matrix[0][route[0]]  # From depot to first node
        for i in range(1, len(route)):
            distance += distance_matrix[route[i-1]][route[i]]
        distance += distance_matrix[route[-1]][0]  # From last node back to depot
    return distance

# Function to calculate total distance of all routes (CVRP)
def total_distance(routes, distance_matrix):
    return sum(calculate_route_distance(route, distance_matrix) for route in routes)


# Generate initial solution using a greedy approach
def greedy_initial_solution():
    routes = [[] for _ in range(num_vehicles)]
    current_load = [0] * num_vehicles
    visited = [False] * num_nodes
    visited[0] = True  # Starting from the depot

    for vehicle in range(num_vehicles):
        current_node = 0
        while True:
            next_node = None
            min_distance = float('inf')
            for i in range(1, num_nodes):
                if not visited[i] and current_load[vehicle] + demands[i] <= vehicle_capacity:
                    if distance_matrix[current_node][i] < min_distance:
                        min_distance = distance_matrix[current_node][i]
                        next_node = i
            if next_node is None:
                break
            routes[vehicle].append(next_node)
            current_load[vehicle] += demands[next_node]
            visited[next_node] = True
            current_node = next_node
    return routes


# 禁忌搜索算法

In [9]:
# a function for checking feasibility
def is_feasible(route, demands, vehicle_capacity):
    total_demand = sum(demands[node] for node in route)
    return total_demand <= vehicle_capacity # True or False

# new algorithm for solving CVRPs with local search 
def generate_neighbors(routes, demands, vehicle_capacity, num_rand_swaps = 10):
    neighbors = []
    for i in range(num_vehicles):
        for j in range(i +1, num_vehicles):
            # Route 1 = [1,2,3], Route 2 = [4,5, 6] -> Swap across routes [2,5]. New route 1: [1,5,3], new route 2: [4,2,6]
            for _ in range(num_rand_swaps):
                new_routes = [route.copy() for route in routes]
                if new_routes[i] and new_routes[j]:
                    node_i = random.choice(new_routes[i])
                    node_j = random.choice(new_routes[j])
                    idx_i = new_routes[i].index(node_i)
                    idx_j = new_routes[j].index(node_j)
                    new_routes[i][idx_i], new_routes[j][idx_j] = new_routes[j][idx_j], new_routes[i][idx_i]

                    # check for feasibility
                    if is_feasible(new_routes[i], demands, vehicle_capacity) and is_feasible(new_routes[j], demands, vehicle_capacity):
                        neighbors.append(new_routes)
    return neighbors

In [7]:
# tabu search

def tabu_search_cvrp(initial_routes, demands, vehicle_capacity, max_iterations = 2000, tabu_size = 20, num_random_swaps = 10):
    best_routes = initial_routes
    best_distance = total_distance(initial_routes, distance_matrix)
    current_routes = initial_routes

    tabu_list = deque(maxlen= tabu_size) # FIFO logic

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_routes, demands, vehicle_capacity, num_random_swaps)
        best_neighbor = None
        best_neighbor_distance = float('inf')

        for neighbor in neighbors:
            neighbor_distance = total_distance(neighbor, distance_matrix)
            if neighbor_distance < best_neighbor_distance and neighbor not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_distance = neighbor_distance

        
        if best_neighbor and best_neighbor_distance < best_distance:
            best_routes = best_neighbor
            best_distance = best_neighbor_distance
            tabu_list.append(best_neighbor)
        
        current_routes = best_neighbor if best_neighbor else current_routes
    
    return best_routes, best_distance

In [10]:
max_iterations = 2000  # 最大迭代次数  
tabu_size = 10         # 禁忌列表大小  
num_random_swaps = 10  # 每轮随机交换的次数

for i in range(len(cvrp_instances)):
    cvrp_instance = cvrp_instances[i]
    vehicle_capacity = cvrp_instance['capacity']
    demands = cvrp_instance['demand']
    distance_matrix = cvrp_instance['edge_weight']
    node_coords = cvrp_instance['node_coord']
    print("=====================",cvrp_instance['comment'],"=====================")
    match = re.search(r"No of trucks: (\d+)", str(cvrp_instance['comment']))  
    if match:  
        num_vehicles = int(match.group(1))  
    num_nodes = len(node_coords)
    print("Number of Nodes:",num_nodes)
    initial_routes = greedy_initial_solution()

    optimized_routes_tabu, optimized_route_distance_tabu = tabu_search_cvrp(initial_routes, demands, vehicle_capacity,
                                                                  max_iterations = max_iterations, tabu_size= tabu_size,
                                                                  num_random_swaps = num_random_swaps)
    
    print("Tabu Optimized Routes: ", optimized_routes_tabu)
    print("Tabue Search Distance: ", optimized_route_distance_tabu)
    
    test_capacity_constraint(optimized_routes_tabu, demands, vehicle_capacity)
    
    #plot_CVRP_solution(optimized_routes_tabu, node_coords)

===================== (Augerat et al, No of trucks: 5, Optimal value: 778) =====================
Number of Nodes: 34
Tabu Optimized Routes:  [[26, 4, 33, 22, 9, 12, 3, 18], [29, 11, 23, 27, 24, 30, 5], [32, 28, 31, 17, 19, 25, 15, 7], [13, 10, 21, 2, 16, 20], [1, 8, 6, 14]]
Tabue Search Distance:  839.0289629142868
Vehicle 1 is within capacity: 100 <= 100
Vehicle 2 is within capacity: 100 <= 100
Vehicle 3 is within capacity: 100 <= 100
Vehicle 4 is within capacity: 75 <= 100
Vehicle 5 is within capacity: 85 <= 100
===================== (Augerat et al, No of trucks: 5, Optimal value: 784) =====================
Number of Nodes: 32
Tabu Optimized Routes:  [[20, 14, 12, 1, 6, 23, 28, 8, 18, 5], [27, 22, 9, 15, 25, 10, 29], [26, 13, 31, 19, 17, 21, 16], [7, 2, 3, 4, 11, 24], [30]]
Tabue Search Distance:  1014.7514788644532
Vehicle 1 is within capacity: 100 <= 100
Vehicle 2 is within capacity: 96 <= 100
Vehicle 3 is within capacity: 100 <= 100
Vehicle 4 is within capacity: 100 <= 100
Vehicle